This script will select bins from binned_sysevents and ..._smear_... that do not have nans.

So there are 4 files that need to have nan bins removed and have same shape

-binned

-smear_theta

-smear_Eh

-smear_El

In [1]:
import numpy as np
import os
import pdb

In [18]:
# select experiment and neutrino
#Homefolder/experiment/binned_sysevents_experiment_neutrino.txt
def get_nonnan_index(file):
    with open(file,'r') as f:
        loc_array = np.array([])
        lines = f.readlines()
        for line_index,line in enumerate(lines):
            if line.split()[2] !='nan':
                loc_array = np.append(loc_array,line_index)
    return loc_array
                
def rewrite(file_read,file_save,loc_array):
    with open(file_read,'r') as f:
        lines = f.readlines()
        write_lines = np.array(lines)[loc_array]
    myfile = open(file_save,'w')
    myfile.writelines(write_lines)

def nanclipper(homefolder,experiment,neutrino,nunubar = 0):
    #make clipped folder
    if nunubar == 1:
        folder_append = 'nunubar'
    else:
        folder_append = str(neutrino)
    clipped_folder = os.path.join(homefolder,experiment,'clipped',folder_append)
    if not os.path.exists(os.path.join(homefolder,experiment,'clipped')):
        os.mkdir(os.path.join(homefolder,experiment,'clipped'))
    print(clipped_folder)
    if not os.path.exists(clipped_folder):
        os.mkdir(clipped_folder)
    
    #get names of 4 files
    filename = 'binned_sysevents_%s_%s.txt' %(experiment,str(neutrino))
    new_file_binned = os.path.join(clipped_folder,'NoNans_'+filename)
    binnedfile = os.path.join(homefolder,experiment,filename)
    
    filename = 'smear_%s_%s_theta.txt' %(experiment,str(neutrino))
    new_file_theta= os.path.join(clipped_folder,'NoNans_'+filename)
    thetafile = os.path.join(homefolder,experiment,filename)

    filename = 'smear_%s_%s_Ehadron.txt' %(experiment,str(neutrino))
    new_file_hadron= os.path.join(clipped_folder,'NoNans_'+filename)
    hadronfile = os.path.join(homefolder,experiment,filename)
   
    filename = 'smear_%s_%s_Elepton.txt' %(experiment,str(neutrino))
    new_file_lepton= os.path.join(clipped_folder,'NoNans_'+filename)
    leptonfile = os.path.join(homefolder,experiment,filename)

    #get locs1 where no nans
    binned_locs = get_nonnan_index(binnedfile).astype('int')
    
    #get locs2
    theta_locs = get_nonnan_index(thetafile).astype('int')
    
    #get locs3
    hadron_locs = get_nonnan_index(hadronfile).astype('int')
    
    #get locs4
    lepton_locs = get_nonnan_index(leptonfile).astype('int')
    
    #get intersection of locs
    locsA = np.intersect1d(binned_locs,theta_locs);locsB = np.intersect1d(hadron_locs,lepton_locs)
    nonnan_locs = np.intersect1d(locsA,locsB)
    
    if nunubar == 0:
        #rewrite file1
        rewrite(binnedfile,new_file_binned,nonnan_locs)

        #rewrite file2
        rewrite(thetafile,new_file_theta,nonnan_locs)

        #rewrite file3 
        rewrite(hadronfile,new_file_hadron,nonnan_locs)

        #rewrite file4
        rewrite(leptonfile,new_file_lepton,nonnan_locs)
    #now write additional script to get anti and neutrino the same
    if nunubar == 1:
        if neutrino < 0:
            raise('Error: do this script with pid of neutrino > 0')
        nonnan_locs_nu = np.copy(nonnan_locs)
        
        #get names of 4 files
        filename = 'binned_sysevents_%s_%s.txt' %(experiment,str(-neutrino))
        new_file_binned_nubar = os.path.join(clipped_folder,'NoNans_'+filename)
        binnedfile_nubar = os.path.join(homefolder,experiment,filename)

        filename = 'smear_%s_%s_theta.txt' %(experiment,str(-neutrino))
        new_file_theta_nubar= os.path.join(clipped_folder,'NoNans_'+filename)
        thetafile_nubar = os.path.join(homefolder,experiment,filename)

        filename = 'smear_%s_%s_Ehadron.txt' %(experiment,str(-neutrino))
        new_file_hadron_nubar= os.path.join(clipped_folder,'NoNans_'+filename)
        hadronfile_nubar = os.path.join(homefolder,experiment,filename)

        filename = 'smear_%s_%s_Elepton.txt' %(experiment,str(-neutrino))
        new_file_lepton_nubar= os.path.join(clipped_folder,'NoNans_'+filename)
        leptonfile_nubar = os.path.join(homefolder,experiment,filename)

        #get locs1 where no nans
        binned_locs = get_nonnan_index(binnedfile_nubar).astype('int')

        #get locs2
        theta_locs = get_nonnan_index(thetafile_nubar).astype('int')

        #get locs3
        hadron_locs = get_nonnan_index(hadronfile_nubar).astype('int')

        #get locs4
        lepton_locs = get_nonnan_index(leptonfile_nubar).astype('int')

        #get intersection of locs
        locsA = np.intersect1d(binned_locs,theta_locs);locsB = np.intersect1d(hadron_locs,lepton_locs)
        nonnan_locs_nubar = np.intersect1d(locsA,locsB)
        nonnan_locs_nunubar = np.intersect1d(nonnan_locs_nu,nonnan_locs_nubar)
        
        #rewrite file1
        rewrite(binnedfile,new_file_binned,nonnan_locs)
        rewrite(binnedfile_nubar,new_file_binned_nubar,nonnan_locs_nunubar)

        #rewrite file2
        rewrite(thetafile,new_file_theta,nonnan_locs)
        rewrite(thetafile_nubar,new_file_theta_nubar,nonnan_locs_nunubar)


        #rewrite file3 
        rewrite(hadronfile,new_file_hadron,nonnan_locs)
        rewrite(hadronfile_nubar,new_file_hadron_nubar,nonnan_locs_nunubar)

        #rewrite file4
        rewrite(leptonfile,new_file_lepton,nonnan_locs)
        rewrite(leptonfile_nubar,new_file_lepton_nubar,nonnan_locs_nunubar)

In [19]:
#clip neutrinos
homefolder,experiment,neutrino = os.getcwd(), 'FASERv2',14
nanclipper(homefolder,experiment,neutrino,nunubar = 0)
#clip antineutrinios
homefolder,experiment,neutrino = os.getcwd(), 'FASERv2',-14
nanclipper(homefolder,experiment,neutrino,nunubar = 0)
#clip both toegther (i.e. nunubarflag =1)
homefolder,experiment,neutrino = os.getcwd(), 'FASERv2',14
nanclipper(homefolder,experiment,neutrino,nunubar = 1)

C:\Users\Mhfie\Downloads\PDFs\PDFs\FASERv2\clipped\nunubar
